In [1]:
!nvidia-smi

Thu Nov  2 16:58:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

# Set the new home directory
new_home_directory = '/content/drive/MyDrive/rataq'
os.environ['HOME'] = new_home_directory

# Check if it's set correctly
HOME = os.environ['HOME']
print("New HOME:", HOME)

# Check the current working directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)
with open("/content/drive/MyDrive/rataq/runs/detect/val/cache.txt", "r") as file:
    content = file.read()

New HOME: /content/drive/MyDrive/rataq
Current Directory: /content


## Install YOLOv8

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.1/78.2 GB disk)


In [4]:
%cd {HOME}

/content/drive/MyDrive/rataq


## Preparing a custom dataset

In [5]:
#!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow



/content/drive/MyDrive/rataq/datasets
  Using cached roboflow-1.1.7-py3-none-any.whl (58 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached opencv_python_headless-4.8.0.74-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (49.1 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Using cached supervision-0.16.0-py3-none-any.whl (72 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless

## Custom Training

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8l.pt data=datasets/data.yaml epochs=25 imgsz=800 plots=True

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 43608921 parameters, 0 gradients, 164.8 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 26/26 [00:30<00:00,  1.19s/it]
                   all        816       1568      0.929      0.953      0.943      0.885
                Cracks        816        563      0.944      0.952      0.950      0.894
               Pothole        816        616      0.966      0.971      0.968      0.932
         BoundaryStone        816        389      0.920      0.941      0.931      0.871
Speed: 0.3ms pre-process, 24.2ms inference, 0.0ms loss, 2.6ms post-process per image
Results saved to runs/detect/train/content/drive/MyDrive/rataq
100% 83.7M/83.7M [00:00<00:00, 180MB/s]

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task

In [ ]:
!ls {HOME}/runs/detect/train/

args.yaml					    results.png		   val_batch0_pred.jpg
confusion_matrix.png				    train_batch0.jpg	   val_batch1_labels.jpg
events.out.tfevents.1698770380.7b51f2533d50.2356.0  train_batch1.jpg	   val_batch1_pred.jpg
F1_curve.png					    train_batch2730.jpg    val_batch2_labels.jpg
P_curve.png					    train_batch2731.jpg    val_batch2_pred.jpg
PR_curve.png					    train_batch2732.jpg    weights
R_curve.png					    train_batch2.jpg
results.csv					    val_batch0_labels.jpg


In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import glob
import os

# Specify the path to the directory containing PNG images
image_directory = os.path.expanduser('~/runs/detect/train/')

# Find all PNG files in the directory
png_files = glob.glob(f'{image_directory}*.png')

# Iterate through the list of PNG files and display them
for image_path in png_files:
    img = cv2.imread(image_path)
    if img is not None:
        cv2_imshow(img)

# No need to call cv2.destroyAllWindows() in Colab


Output hidden; open in https://colab.research.google.com to view.

## Inference with Custom Model

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={HOME}/datasets/test/images save=True

**NOTE:** Let's take a look at few results.

In [ ]:
!ls {HOME}/runs/detect/predict

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import glob
import os

# Specify the path to the directory containing JPG images
image_directory = os.path.expanduser('~/runs/detect/predict/')

# Find all JPG files in the directory
jpg_files = glob.glob(f'{image_directory}*.jpg')

# Iterate through the list of JPG files and display them
for image_path in jpg_files:
    img = cv2.imread(image_path)
    if img is not None:
        cv2_imshow(img)

# No need to call cv2.destroyAllWindows() in Colab


Output hidden; open in https://colab.research.google.com to view.